Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [5]:
from __future__ import print_function
import os

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [6]:
DATA_DIR = '/Users/gleb/Data/udacity-tensorflow-data'

First reload the data we generated in _notmist.ipynb_.

In [7]:
pickle_file = os.path.join(DATA_DIR, 'notMNIST.pickle')

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
batch_size = 50

hidden_1_num_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_1_num_nodes]))
    biases_1 = tf.Variable(tf.zeros([hidden_1_num_nodes]))
    
    hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
    weights_2 = tf.Variable(
        tf.truncated_normal([hidden_1_num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_1, weights_2) + biases_2
    loss = (tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                    logits, tf_train_labels)))
    
    reg_constant = 5e-4
    reg = reg_constant * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
    loss += reg

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
            tf.matmul(
                    tf.nn.relu(
                            tf.matmul(tf_valid_dataset, weights_1) + biases_1),
                    weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
            tf.matmul(
                    tf.nn.relu(
                            tf.matmul(tf_test_dataset, weights_1) + biases_1),
                    weights_2) + biases_2)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 523.424438
Minibatch accuracy: 12.0%
Validation accuracy: 28.2%
Minibatch loss at step 100: 197.184082
Minibatch accuracy: 74.0%
Validation accuracy: 74.8%
Minibatch loss at step 200: 215.007538
Minibatch accuracy: 78.0%
Validation accuracy: 75.7%
Minibatch loss at step 300: 148.830688
Minibatch accuracy: 74.0%
Validation accuracy: 76.6%
Minibatch loss at step 400: 149.841583
Minibatch accuracy: 80.0%
Validation accuracy: 75.7%
Minibatch loss at step 500: 133.683380
Minibatch accuracy: 90.0%
Validation accuracy: 75.4%
Minibatch loss at step 600: 138.833710
Minibatch accuracy: 74.0%
Validation accuracy: 77.3%
Minibatch loss at step 700: 115.760597
Minibatch accuracy: 78.0%
Validation accuracy: 74.4%
Minibatch loss at step 800: 111.036217
Minibatch accuracy: 84.0%
Validation accuracy: 74.4%
Minibatch loss at step 900: 102.051033
Minibatch accuracy: 78.0%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 97.652756
Minibatch accuracy: 70.0%
Valid

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
batch_size = 50

hidden_1_num_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_1_num_nodes]))
    biases_1 = tf.Variable(tf.zeros([hidden_1_num_nodes]))
    
    hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
    weights_2 = tf.Variable(
        tf.truncated_normal([hidden_1_num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_1, weights_2) + biases_2
    loss = (tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                    logits, tf_train_labels)))
    
    reg_constant = 5e-4
    reg = reg_constant * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
    loss += reg

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
            tf.matmul(
                    tf.nn.relu(
                            tf.matmul(tf_valid_dataset, weights_1) + biases_1),
                    weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
            tf.matmul(
                    tf.nn.relu(
                            tf.matmul(tf_test_dataset, weights_1) + biases_1),
                    weights_2) + biases_2)

num_steps = 500

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Always same offset to force over-fitting.
    offset = 0

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 539.433044
Minibatch accuracy: 14.0%
Validation accuracy: 25.1%
Minibatch loss at step 100: 150.192078
Minibatch accuracy: 100.0%
Validation accuracy: 50.8%
Minibatch loss at step 200: 142.866165
Minibatch accuracy: 100.0%
Validation accuracy: 50.8%
Minibatch loss at step 300: 135.897751
Minibatch accuracy: 100.0%
Validation accuracy: 50.9%
Minibatch loss at step 400: 129.269180
Minibatch accuracy: 100.0%
Validation accuracy: 50.9%
Test accuracy: 55.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
batch_size = 128

hidden_1_num_nodes = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_1_num_nodes]))
    biases_1 = tf.Variable(tf.zeros([hidden_1_num_nodes]))
    
    hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
    keep_prob = tf.placeholder(tf.float32)
    hidden_1_dropout = tf.nn.dropout(hidden_1, keep_prob)
    
    weights_2 = tf.Variable(
        tf.truncated_normal([hidden_1_num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_1_dropout, weights_2) + biases_2
    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                    logits, tf_train_labels))
    
    reg_constant = 5e-4
    reg = reg_constant * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
    loss += reg

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
            tf.matmul(
                    tf.nn.relu(
                            tf.matmul(tf_valid_dataset, weights_1) + biases_1),
                    weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
            tf.matmul(
                    tf.nn.relu(
                            tf.matmul(tf_test_dataset, weights_1) + biases_1),
                    weights_2) + biases_2)

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # Always same offset to force over-fitting.
        # offset = 0

        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels : batch_labels,
            keep_prob: 0.5
        }

        # Run an evaluation step.
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        # Sample the current strength of the neural net using the validation set.
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
    
    # Output the final test prediction.  
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 640.766724
Minibatch accuracy: 11.7%
Validation accuracy: 38.2%
Minibatch loss at step 100: 247.252777
Minibatch accuracy: 73.4%
Validation accuracy: 78.2%
Minibatch loss at step 200: 278.974121
Minibatch accuracy: 67.2%
Validation accuracy: 72.9%
Minibatch loss at step 300: 147.296936
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 400: 153.968018
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 500: 149.267441
Minibatch accuracy: 78.1%
Validation accuracy: 80.7%
Minibatch loss at step 600: 137.799347
Minibatch accuracy: 73.4%
Validation accuracy: 78.8%
Minibatch loss at step 700: 129.460403
Minibatch accuracy: 71.9%
Validation accuracy: 80.7%
Minibatch loss at step 800: 114.089279
Minibatch accuracy: 75.0%
Validation accuracy: 80.9%
Minibatch loss at step 900: 114.642532
Minibatch accuracy: 74.2%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 101.640709
Minibatch accuracy: 63.3%
Vali

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [32]:
batch_size = 24

hidden_1_num_nodes = 1024

hidden_2_num_nodes = 1024

INITIAL_LEARNING_RATE = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
            shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    keep_prob = tf.placeholder(tf.float32)
    
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_1_num_nodes]))
    biases_1 = tf.Variable(tf.zeros([hidden_1_num_nodes]))
    
    weights_hidden_1_to_hidden_2 = tf.Variable(
        tf.truncated_normal([hidden_1_num_nodes, hidden_2_num_nodes]))
    biases_hidden_1_to_hidden_2 = tf.Variable(tf.zeros([hidden_2_num_nodes]))
    
    weights_hidden_2_to_logits = tf.Variable(
        tf.truncated_normal([hidden_2_num_nodes, num_labels]))
    biases_hidden_2_to_logits = tf.Variable(tf.zeros([num_labels]))
    
    def build_common_graph(input_data):
        """Function that builds the core inner network.

        Allows making sure that train, validate, and test are really
        using the same network.

        Returns handle to the output layer.
        """
        hidden_1 = tf.nn.relu(
                tf.matmul(input_data, weights_1) + biases_1)
        hidden_1_dropout = tf.nn.dropout(hidden_1, keep_prob)
            
#         hidden_2 = tf.nn.relu(
#                 tf.matmul(hidden_1_dropout, weights_hidden_1_to_hidden_2) + 
#                 biases_hidden_1_to_hidden_2)
#         hidden_2_dropout = tf.nn.dropout(hidden_2, keep_prob)

        hidden_2 = tf.nn.relu(
                tf.matmul(hidden_1, weights_hidden_1_to_hidden_2) + 
                biases_hidden_1_to_hidden_2)
        
        logits = (
                tf.matmul(hidden_2, weights_hidden_2_to_logits) +
                biases_hidden_2_to_logits)

        return logits

    # Training computation.
    logits = build_common_graph(tf_train_dataset)

    loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                    logits, tf_train_labels))
    
    #   Add regularization for training.
    reg_constant = 5e-4
    reg = reg_constant * (
            tf.nn.l2_loss(weights_1) +
            tf.nn.l2_loss(weights_hidden_1_to_hidden_2) +
            tf.nn.l2_loss(weights_hidden_2_to_logits)
    )
    loss += reg

    # Optimizer, constant learning rate.
    # optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(loss)
    
    # Optimizer with decaying learning rate.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(INITIAL_LEARNING_RATE, global_step, 100000, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    valid_prediction = tf.nn.softmax(build_common_graph(tf_valid_dataset))

    test_prediction = tf.nn.softmax(build_common_graph(tf_test_dataset))

num_steps = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels : batch_labels,
            keep_prob: 0.5
        }

        # Run an evaluation step.
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        # Sample the current strength of the neural net using the validation set.
        if (step % 10 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
    
            # Output the final test prediction.  
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 6910.920898
Minibatch accuracy: 4.2%
Validation accuracy: 12.3%
Test accuracy: 12.2%
Minibatch loss at step 10: 3534.276855
Minibatch accuracy: 33.3%
Validation accuracy: 42.4%
Test accuracy: 46.4%
Minibatch loss at step 20: 1806.539551
Minibatch accuracy: 62.5%
Validation accuracy: 57.6%
Test accuracy: 62.6%
Minibatch loss at step 30: 1502.477783
Minibatch accuracy: 62.5%
Validation accuracy: 58.7%
Test accuracy: 63.4%
Minibatch loss at step 40: 3278.854492
Minibatch accuracy: 33.3%
Validation accuracy: 50.9%
Test accuracy: 55.8%
Minibatch loss at step 50: 653.090698
Minibatch accuracy: 87.5%
Validation accuracy: 67.0%
Test accuracy: 73.2%
Minibatch loss at step 60: 1288.936768
Minibatch accuracy: 79.2%
Validation accuracy: 69.7%
Test accuracy: 75.6%
Minibatch loss at step 70: 722.646851
Minibatch accuracy: 75.0%
Validation accuracy: 70.8%
Test accuracy: 76.6%
Minibatch loss at step 80: 908.158325
Minibatch accuracy: 70.8%
Validation accuracy: 71.